### Extraction of GPS in excel from mdb's
* Reading mdbs from floder and convert into excel sheet
* Print the the number of mdb present in a floder which converted into excel

In [4]:
import pyodbc,os,pandas as pd

mdb_directory = r"D:\_2.Programming\_6.test\mdb_processed" # input directory
o_gps = r"D:\_2.Programming\_6.test\Ex_GPS.xlsx" # output directory

#defining function to read mdb's
def read_mdb(x):
    mdb_files = []
    for root, dirs, files in os.walk(x):
        for file in files:
            if file.endswith(".mdb"):
                mdb_files.append(os.path.join(root, file))
    return mdb_files

#defining function to read tables in mdb
def read_tables_by_prefix(mdb_file, table_prefix):
    conn_str = f"Driver={{Microsoft Access Driver (*.mdb, *.accdb)}};Dbq={mdb_file};"
    connection = pyodbc.connect(conn_str)
    cursor = connection.cursor()

    tables = []
    for table_info in cursor.tables(tableType='TABLE'):
        table_name = table_info.table_name
        if table_name.startswith(table_prefix):
            tables.append(table_name)

    data = pd.DataFrame()
    for table in tables:
        query = f"SELECT * FROM [{table}]"
        table_data = pd.read_sql(query, connection)
        table_data['Survey_ID_1'] = os.path.splitext(os.path.basename(mdb_file))[0]
        data = pd.concat([data, table_data], ignore_index=True)

    connection.close()
    return data

#calling function to read mdb's
mdb_files = read_mdb(mdb_directory)

#pandas data frame for storing extracted gps and keycode
GPS_P = pd.DataFrame()

#extracting and merging gps and keycode from mdb's 
for mdb_file in mdb_files:
    gps = read_tables_by_prefix(mdb_file, "GPS_Processed")
    GPS_P = pd.concat([GPS_P, gps], ignore_index=True)

#exporting gps and keycode from pandas data frame
GPS_P.to_excel(o_gps, index=False)

print(len(mdb_files))
print("Successfully completed!!")


C:\Users\asifi\AppData\Local\Temp\ipykernel_24348\1907247421.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_data = pd.read_sql(query, connection)
C:\Users\asifi\AppData\Local\Temp\ipykernel_24348\1907247421.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_data = pd.read_sql(query, connection)
C:\Users\asifi\AppData\Local\Temp\ipykernel_24348\1907247421.py:30: UserWarning: pandas only supports SQLAlchemy connectable (engine/connection) or database string URI or sqlite3 DBAPI2 connection. Other DBAPI2 objects are not tested. Please consider using SQLAlchemy.
  table_data = pd.read_sql(query, connection)
C:\Users\asifi\AppData\Local\Temp\ipykernel_24348\19072

6
Successfully completed!!


### Creating feature classes from excel data
* Excel to table 
* Table to point 
* Point to line in WGS & UTM
* Adding schema

In [ ]:
in_gps = r"D:\_2.Programming\_6.test\Ex_GPS.xlsx" # input directory
arcpy.env.overwriteOutput = 1
arcpy.env.workspace = r"D:\_2.Programming\_6.test\test.gdb" # workspace
arcpy.ExcelToTable_conversion(in_gps, "T_GPS", "Sheet1")
arcpy.management.XYTableToPoint("T_GPS","P_GPS","LONGITUDE","LATITUDE","",arcpy.SpatialReference(4326))
arcpy.management.PointsToLine("P_GPS", "L_GPS", "Survey_ID_1") # L_GPS is in WGS-84
arcpy.management.AddFields("L_GPS",[["STREET_NAME","TEXT"],["STREET_ID","TEXT"],["IP_IP","TEXT"], ["DIRECTION","TEXT"],
                                            ["From_","TEXT"],["To","TEXT"],["Length","DOUBLE"],["SECTION_NO","DOUBLE"]])
utm_crs = arcpy.SpatialReference(32640)
arcpy.management.Project("L_GPS","L_GPS_utm",utm_crs)

### Flip line based on direction

In [ ]:
arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
field = 'DIRECTION' # specify field name based on which flip will undergo
infc = 'L_GPS_utm' # specify feature class
def flip_line_geometry(line): # Define a function to flip line geometry
    reversed_coords = [(point.X, point.Y) for point in reversed(line.getPart(0))]
    return arcpy.Polyline(arcpy.Array([arcpy.Point(x, y) for x, y in reversed_coords]))
with arcpy.da.UpdateCursor(infc, ["SHAPE@", field]) as cursor: # Update the geometry of lines that meet the condition
    for row in cursor:
        direction = row[1]
        if direction == "Dec":
            line_geometry = row[0]
            flipped_line = flip_line_geometry(line_geometry)
            row[0] = flipped_line
            cursor.updateRow(row)

print("Successfully completed!!!")

### Creating points from GPS line
* points interval is dynamic 

In [ ]:
import arcpy
import math
import os

arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
arcpy.env.overwriteOutput = True

in_Fc = 'L_GPS_utm'
ofc_Points = 'points_100m'
interval_distance = 100


# Add fields to store the interval number, Unique_ID, and Line_ID
arcpy.CreateFeatureclass_management(out_path=os.path.dirname(ofc_Points),out_name=os.path.basename(ofc_Points),geometry_type="POINT",spatial_reference= in_Fc)

arcpy.AddFields_management(ofc_Points, [["Interval", "LONG"],["Chainage", "LONG"],["Section_No", "LONG"]])

# Start an insert cursor
with arcpy.da.InsertCursor(ofc_Points, ["SHAPE@", "Interval", "Chainage", "Section_No"]) as cursor:
    interval = 0    
    with arcpy.da.SearchCursor(in_Fc, ["SHAPE@"]) as search_cursor:
        for row in search_cursor:
            line = row[0]
            length = line.length  # Get the length of the line
            i = 0
            chainage = 0  # Initialize the Unique_ID for the current line
            sec_no = 1
            while i < length:
                point = line.positionAlongLine(i)
                cursor.insertRow([point, interval, chainage, sec_no])
                interval += 1
                chainage += 100
                sec_no += 1
                i += interval_distance   
print ("sucessfully created points")

### Splitting pms lines on 100m of sections

In [ ]:
import arcpy

arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
arcpy.env.overwriteOutput = True

in_Fc = 'L_GPS_utm'
in_Point = 'points_100m'
out_Fc = 'PMS_Sections'

radius = 0.5

# Split the line at points using arcpy.SplitLineAtPoint_management
arcpy.SplitLineAtPoint_management(in_Fc, in_Point, out_Fc, radius)

# Create a dictionary to store the cumulative lengths for each line
cumulative_lengths = {}

# Use an update cursor to calculate and populate the "FromValue" field
with arcpy.da.UpdateCursor(out_Fc, ["SHAPE@", "OID@"]) as cursor:
    for row in cursor:
        line_geometry = row[0]
        oid = row[1]
        
        # Check if the geometry is multipart
        if line_geometry.isMultipart:
            # If multipart, iterate over parts and update each part separately
            cumulative_length = 0.0
            for part in line_geometry:
                cumulative_length += part.length
        else:
            cumulative_length = line_geometry.length
        
        # Update the cumulative length for this line
        cumulative_lengths[oid] = cumulative_length



### Populating attributes
* From,To,section_no and length fields are populated

In [ ]:
import arcpy

arcpy.env.workspace = r'D:\_2.Programming\_6.test\test.gdb'
arcpy.env.overwriteOutput = True

in_Fc = 'PMS_Sections'
in_Points = 'points_100m'

# Create a dictionary to store point coordinates and their corresponding Unique_IDs
point_dict = {}
to_dict = {}

with arcpy.da.SearchCursor(in_Points, ["SHAPE@", "Chainage", "Section_No"]) as cursor:
    for sc in cursor:
        point = sc[0].firstPoint  # Get the first point of the geometry
        chainage = sc[1]
        section_no = sc[2]
        point_coords = (point.X, point.Y)  # Convert point to a tuple of coordinates
        point_dict[point_coords] = chainage
        to_dict[point_coords] = section_no

# Update the "FROM", "TO", and "LENGTH" fields in PMS_Sections
with arcpy.da.UpdateCursor(in_Fc, ["SHAPE@", "From_", "SECTION_NO","To", "Length"]) as up_cursor:
    for up in up_cursor:
        line_geometry = up[0]
        first_point = (line_geometry.firstPoint.X, line_geometry.firstPoint.Y)  # Convert to tuple
        
        # Check if the exact coordinate match exists
        if first_point in point_dict:
            up[1] = point_dict[first_point]
            up[3] = up[1] + round(up[0].length,0)
            up[4] = round(up[0].length,0)
            up_cursor.updateRow(up)
        else:
            # If no exact match, find the nearest point
            nearest_point = min(point_dict.keys(), key=lambda p: arcpy.PointGeometry(arcpy.Point(*p)).distanceTo(line_geometry.firstPoint))
            up[1] = point_dict[nearest_point]
            up_cursor.updateRow(up)

        # Similar logic for the "TO" field
        if first_point in to_dict:
            up[2] = to_dict[first_point]
            up_cursor.updateRow(up)
        else:
            # If no exact match, find the nearest point
            nearest_point = min(to_dict.keys(), key=lambda p: arcpy.PointGeometry(arcpy.Point(*p)).distanceTo(line_geometry.firstPoint))
            up[2] = to_dict[nearest_point]
            up_cursor.updateRow(up)

print("Successfully populated FROM, TO, and LENGTH fields.")
